In [37]:
from bnn.model import load_baysean_model, load_full_model, encoder_min_input_size
from bnn import evaluate
from bnn.data import test_train_batch_data, test_train_data
from bnn.util import BayesianConfig

from keras import backend as K
import numpy as np

In [3]:
encoder = 'resnet50'
dataset = 'cifar10'
debug = True
monte_carlo_simulations = 100
min_image_size = (197,197,3)
full_model = True


In [4]:
((x_train, y_train), (x_test, y_test)) = test_train_data(dataset, min_image_size[0:2], debug)

Resizing images from (32, 32) to (197, 197)
Done resizing train images.
Done resizing test images.


In [6]:
config = BayesianConfig(encoder, dataset, 256, 50, monte_carlo_simulations)
model = evaluate.load_testable_model(encoder, config, monte_carlo_simulations, y_train.shape[-1], min_image_size, full_model)

/Users/kyledorman/anaconda/lib/python3.5/site-packages/keras/models.py:268: UserWarning: Output "softmax_output" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "softmax_output" during training.
  sample_weight_mode=sample_weight_mode)


Compiling model.


/Users/kyledorman/Documents/carnd/bayesian-neural-network-blogpost/bnn/evaluate.py:49: UserWarning: Output "softmax_output" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "softmax_output" during training.
  metrics={'softmax_output': 'categorical_accuracy', 'softmax_output': 'top_k_categorical_accuracy'})


In [24]:
predictions = model.predict(x_train[0:128])
labels = y_train[0:128]

In [32]:
uncertainties = predictions[0][:,10:]

In [43]:
from tensorflow.contrib import distributions
std = K.sqrt(K.variable(uncertainties))
dist = distributions.Normal(loc=K.zeros_like(a), scale=a)
K.eval(dist.sample(20))

array([[ 0.11820246,  0.75832768],
       [ 0.43192474,  0.0192488 ],
       [ 0.8923598 ,  0.1803694 ],
       [ 0.55716212,  0.01950079],
       [ 3.80515113,  0.22466248],
       [-0.93007683,  1.66259945],
       [-0.61921775, -1.32401168],
       [-0.99122913, -1.06474315],
       [ 0.19741622, -0.2971317 ],
       [-0.59747733,  2.04673066],
       [ 1.00084328, -2.06180167],
       [-0.71037481,  1.50037795],
       [ 0.85387373, -0.74232105],
       [-0.82249384, -1.32007156],
       [ 0.36328592, -3.61078289],
       [ 0.22860899, -0.39018346],
       [ 1.39246963,  1.01088665],
       [ 0.88803769, -3.41080758],
       [-0.25744589,  4.46698222],
       [-0.68879473, -0.85009728]])

In [23]:
print(np.argmax(predictions, axis=1)-np.argmax(labels, axis=1))

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  2  0  0  0  1  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  4  0 -3  0  0 -2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0 -8  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0
  0  0  0]
